# Title: Test Book for Betakit Funding page scrape

Step by step process:
- get themes from each criteria, by finance activity mentioned (ie. acqusition, rasing capital, private debt, etc.)
- Look at each text block combined, organized with SET(Date, Title & Description Combined), and search for those theme
- Create a list of HREFs that match the list, then open each up and call the page.
- For each HREF, we add the page content HTML to the SET(), for each of the SET, we will chunk it by matching if the text exisits inside the prargraph block, and only keep those
- We use each of the relevatn blcoks to create a summary, then we output it. Key distinct is that if there is no numbers, it will be considered an undiclosed deal
- we compile an HTML emaile and set it out to the respective email

Segments of trial for betakit specifically:
1. Pull down betakit funding website
    a. Sort into: SET(date in datetime, title+description combined, href for article link)
2.  Regex search (for ease, insteal of embedding)
    a. [Will be given by AI] Search for acquisitions / funding inside the title+description, put the ones that match into another list
    b. Scrap each HTML for the HREF and add it to the set 


### 0: Set up (imports)

In [46]:
# poetry add requests
# poetry add bs4
# poetry add html5lib -> in #1
import requests
from bs4 import BeautifulSoup as bs
import re
import datetime

### 1: Pull down betakit funding website
Sort into: SET(date in datetime, title+description combined, href for article link)

1. Read github of webscraper: https://github.com/dirkjbreeuwer/gpt-automated-web-scraper/blob/main/website_analysis/dom_analysis.py#L36
Learnings:
- first you parse raw HTML using simple bs4 BeautifulSoup and 'html.parser' -> this extracted object can use regex search
- then this is a soup, in which you can use something like: soup(text=re.compile(target_string, re.I)) for case sensitive extraction of element
- in html, you want more than just the text, so in the github, he used 3 .parent code to ensure most of the surrounding code is included

In [26]:
# Funding Site: https://betakit.com/tag/funding/
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"} 
betakit_response = requests.get(url="https://betakit.com/tag/funding/", headers=headers)
# soup = bs(betakit_response.content, 'html5lib') #library parser needed
soup = bs(betakit_response.content, 'html.parser') # this also works
print(soup.prettify()) 

<!DOCTYPE html>
<html dir="ltr" lang="en-CA" prefix="og: https://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, minimum-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <title>
   Funding | BetaKit
  </title>
  <!-- All in One SEO Pro 4.6.1 - aioseo.com -->
  <meta content="max-image-preview:large" name="robots">
   <meta content="uniting the prairies,dan vandal,prairiescan,saskatchewan,alberta,manitoba,prairies,co.labs,conexus venture capital,conexus credit union,regina,saskatoon,software,jordan mcfarlen,sean o’connor,kyle scott,celina philpot,katherine regnier,coconut software,7shifts,québec,espace aéro,aerospace,boeing,pierre fitzgibbon,héroux-devtek,wisk,aéro montréal,longueuil,mirabel,collaborative center for innovation in aerospace and mobility,airbus,bombardier,innovation zones,beacon,paytm,psp investments,stuart szabo,aditya mhatre,wayne pommen,jawl residential 

In [38]:
# search class: "grids list-layout entries" as there is only one, and here is the list of the article on the funding page
#docs: https://tedboy.github.io/bs4_doc/generated/generated/bs4.Tag.html
mydivs = soup.find_all("div", {"class": "grids list-layout entries"}) # there should only be 1
# print(mydivs)
# mydivs[0] -> <class 'bs4.element.Tag'>
lst = mydivs[0].findChildren("article")
print(len(lst))

15


In [89]:
# Place into structured list

# Sample item index 0
item_one = lst[0]
# print(item_one)
# print('==================')
raw_date = item_one.find('span', {'class': 'entry-date'})
# https://www.w3schools.com/python/python_datetime.asp
date_formated = datetime.datetime.strptime(raw_date.text, '%b %d, %Y')
# is there a way to make this flexible to changes?
title = item_one.find('h2', {'class': 'entry-title'}).findChild('a').text
desc = item_one.find('div', {'class': 'entry-summary'}).findChild('p').text
combo_text = title + '; ' + desc
article_href = item_one.find('h2', {'class': 'entry-title'}).findChild('a').get('href')
print(f'Date: {date_formated.isoformat()}\n', f'Combo: {combo_text}\n', f'Href: {article_href}\n')
tup_1 = (date_formated, combo_text, article_href)

target_string = 'invest'

Date: 2024-05-23T00:00:00
 Combo: Federal government invests $2.5 million in Co.Labs to maintain Uniting the Prairies; The funding will help stage the Uniting the Prairies conference for the next three years. 
 Href: https://betakit.com/federal-government-invests-2-5-million-in-co-labs-to-maintain-uniting-the-prairies/



### 2: Regex search (for ease, insteal of embedding)
    a. [Will be given by AI] Search for acquisitions / funding inside the title+description, put the ones that match into another list
    b. Scrap each HTML for the HREF and add it to the set 



In [105]:
# Take tup_1, and assume this is what we want to use.
# we will extract the content, then regex find relevant elements
tup_1_response = requests.get(url=tup_1[2], headers=headers)
soup_tup_1 = bs(tup_1_response.content, 'html.parser')
elements = soup_tup_1(text=re.compile(target_string, re.I))
filtered_elements = [str(element).strip() for element in elements if element.parent.name not in ['script', 'title']] #this is betakit specific
# print(len(elements))
# print(elements[1].parent)
# filtered_elements
filtered_elements

['Federal government invests $2.5 million in Co.Labs to maintain Uniting the Prairies',
 '. The Co.Labs-established event brings together investors, startups, and community members from across Saskatchewan, Alberta, and Manitoba.',
 'Since it launched in 2017, Co.Labs has incubated 201 Saskatchewan tech start-ups, raised over $37 million in private investment, generated over $60 million in revenue, and created over 800 jobs, according to the federal government.',
 'for its early-stage tech investor tax credit, the Saskatchewan Technology Startup Incentive (STSI), from $3.5 million to $7 million and expanded it to include cleantech startups as eligible businesses. The STSI offers a 45 percent tax credit to individuals who invest in eligible tech startups in the province.',
 'its second $30-million CAD VC fund with a plan to invest in more early-stage Saskatchewan-based technology startups.',
 '0 replies on “Federal government invests $2.5 million in Co.Labs to maintain Uniting the Prair

### Other

In [9]:
# Element search
target_string = 'funding'
result_elements = soup(text=re.compile(target_string, re.I))
print(result_elements)

['Funding | BetaKit', '\n\t\t\t{"@context":"https:\\/\\/schema.org","@graph":[{"@type":"BreadcrumbList","@id":"https:\\/\\/betakit.com\\/tag\\/funding\\/#breadcrumblist","itemListElement":[{"@type":"ListItem","@id":"https:\\/\\/betakit.com\\/#listItem","position":1,"name":"Home","item":"https:\\/\\/betakit.com\\/","nextItem":"https:\\/\\/betakit.com\\/tag\\/funding\\/#listItem"},{"@type":"ListItem","@id":"https:\\/\\/betakit.com\\/tag\\/funding\\/#listItem","position":2,"name":"Funding","previousItem":"https:\\/\\/betakit.com\\/#listItem"}]},{"@type":"CollectionPage","@id":"https:\\/\\/betakit.com\\/tag\\/funding\\/#collectionpage","url":"https:\\/\\/betakit.com\\/tag\\/funding\\/","name":"Funding | BetaKit","inLanguage":"en-CA","isPartOf":{"@id":"https:\\/\\/betakit.com\\/#website"},"breadcrumb":{"@id":"https:\\/\\/betakit.com\\/tag\\/funding\\/#breadcrumblist"}},{"@type":"Organization","@id":"https:\\/\\/betakit.com\\/#organization","name":"Betakit Inc.","url":"https:\\/\\/betakit.co

In [14]:
result_elements[1].parent.parent

<meta content="All in One SEO Pro (AIOSEO) 4.6.1" name="generator">
<meta content="en_US" property="og:locale"/>
<meta content="BetaKit" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="Funding | BetaKit" property="og:title"/>
<meta content="https://betakit.com/tag/funding/" property="og:url"/>
<meta content="https://www.facebook.com/BetaKit/" property="article:publisher"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@betakit" name="twitter:site"/>
<meta content="Funding | BetaKit" name="twitter:title"/>
<script class="aioseo-schema" type="application/ld+json">
			{"@context":"https:\/\/schema.org","@graph":[{"@type":"BreadcrumbList","@id":"https:\/\/betakit.com\/tag\/funding\/#breadcrumblist","itemListElement":[{"@type":"ListItem","@id":"https:\/\/betakit.com\/#listItem","position":1,"name":"Home","item":"https:\/\/betakit.com\/","nextItem":"https:\/\/betakit.com\/tag\/funding\/#listItem"},{"@type":"ListItem","@id":"htt